Network Compression:smaller model less parameters
理由：resource-constrained environments
为什么不上云? Lower latency,Privacy,etc.
这里只考虑:软体压缩 不考虑硬体加速
Network Pruning:Network can be pruned   1989年就提出了Optimal Brain Damage 这篇paper
framework:Pre-trained Network很大,Evaluate the importance of a weight(absolute values,life long...)/neuron(the number of times it wasn't zero on a given data set...),之后Remove就得到了比较小的Network,这样accuracy就会掉一点,然后我们你需要Fine-tune一下让他的accuracy上升回来.还可以重新评估一次,再Remove,重新Fine-tune.  tips:一次只剪掉一点
Network Pruning - Practical Issue
Weight pruning:剪掉之后的network是不规则的,导致hard to implement,hard to speedup,不好实做.因为input和output不好写vector,GPU加速也不好加,因为矩阵的乘法不容易.所以Prune掉的weight直接补零.这样的问题是没有真正地把Network变小,因为这样参数还是存在,只不过为零.多数情况下都是变慢的.
Neuron pruning:network剪完后是规则的.

大的network比较好trian(Lottery Ticket Hypothesis)
大的network可以看成许多小的network,只要有一个小的network,大的network就成功.(大乐透假说)
直接训练小的,train不起来.
解构大乐透.不同的策略是不同的效果.不改变参数的正负号是关键,绝对值不重要.随机初始化一下得到了好的结果,不用train,直接prune就可以(Weight Agnostic Neural Networks也有类似的结论)

Knowledge Distillation
先训练一个大的Network叫Teacher Net,根据这个训练出来的小的叫Student Net(修剪).
Student Net的输出要接近 Teacher Net的输出(Learning target).不直接trian小的network的原因和上面一样.因为往往效果不怎么样.Student Net可能没有看过input,但也会.
Teacher Net 可以是多个Network的Ensemble.Ensemble:Average many model.

Temperature for softmax
作用:让比较集中的分布变得比较平滑.让学生学到数据之间的关系,学的比较好.
Parameter Quantization
1.用少的空间储存比较多的参数
2.把相近的数值分群,然后把每个群里面取出平均值.
3.常出现的用少的bit描述,罕见的用比较多的bit描述 e.g.Huffman encoding
Binary Weights(效果居然比较好,还可以达到防止overfitting的作用)


Architecture Design
*Depthwise Separable Convolution*
Review: Standard CNN
Input:feature map
filter的数量 = output的feature map

一般的CNN的Input和output的channel是不一样的,但是在Depthwise Convolution里面是一样的
1.Depthwise Convolution 同一个channel内部的关系
Filter number = Input channel number 每个filter只负责一个Input channel 这里的filter没有厚度

2.Pointwise Convolution 输入和输出之间的channel可以不一样,只需要考虑channel之间的关系
Kernel Size 都是1 x 1

Low rank approximation
把一层network拆成两层network,对参数的需求减小了.但是W中的参数受限制

Dynamic Computation
前几个方法是把Network变小,但再Dynamic Computation可以自由调整运算量.
让network自由调整深度.训练的时候把每个extra layer的output和ground truth越接近越好,把所有的output的ground truth的距离加起来得到L,minimize L.

让network自由决定宽度.和上面的一样.

Computation based on Sample Difficulty

这几个压缩是可以一起用的,不互斥.

In [1]:
# Import some useful packages for this homework
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset # "ConcatDataset" and "Subset" are possibly useful
from torchvision.datasets import DatasetFolder, VisionDataset
from torchsummary import summary
from tqdm.auto import tqdm
import random

# !nvidia-smi # list your current GPU

In [15]:
cfg = {
    'dataset_root': '/kaggle/input/homework13-dataset/Food-11',
    'save_dir': '/kaggle/working/',
    'exp_name': "simple_baseline",
    'batch_size': 64,
    'lr': 3e-4,
    'seed': 20220013,
    'loss_fn_type': 'KD', # simple baseline: CE, medium baseline: KD. See the Knowledge_Distillation part for more information.
    'weight_decay': 1e-5,
    'grad_norm_max': 10,
    'n_epochs': 3, # train more steps to pass the medium baseline.
    'patience': 300,
}

In [18]:
myseed = cfg['seed']  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
random.seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

save_path = os.path.join(cfg['save_dir'], cfg['exp_name']) # create saving directory
os.makedirs(save_path, exist_ok=True)

# define simple logging functionality
log_fw = open(f"{save_path}/log.txt", 'w') # open log file to save log outputs
def log(text):     # define a logging function to trace the training process
    print(text)
    log_fw.write(str(text)+'\n')
    log_fw.flush()

log(cfg)  # log your configs to the log file

{'dataset_root': '/kaggle/input/homework13-dataset/Food-11', 'save_dir': '/kaggle/working/', 'exp_name': 'simple_baseline', 'batch_size': 64, 'lr': 0.0003, 'seed': 20220013, 'loss_fn_type': 'KD', 'weight_decay': 1e-05, 'grad_norm_max': 10, 'n_epochs': 3, 'patience': 300}


In [19]:
for dirname, _, filenames in os.walk('/kaggle/input/homework13-dataset/Food-11'):
    if len(filenames) > 0:
        print(f"{dirname}: {len(filenames)} files.") # Show the file amounts in each split.

/kaggle/input/homework13-dataset/Food-11: 1 files.
/kaggle/input/homework13-dataset/Food-11/validation: 4432 files.
/kaggle/input/homework13-dataset/Food-11/training: 9993 files.
/kaggle/input/homework13-dataset/Food-11/evaluation: 2218 files.


In [20]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# define training/testing transforms
test_tfm = transforms.Compose([
    # It is not encouraged to modify this part if you are using the provided teacher model. This transform is stardard and good enough for testing.
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

train_tfm = transforms.Compose([
    transforms.Resize(256),          # 先缩放到256像素
    
    # 增强的数据增强组合
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 随机缩放裁剪(保持224大小)
    transforms.RandomHorizontalFlip(p=0.5),               # 50%概率水平翻转
    transforms.ColorJitter(
        brightness=0.2,             # 亮度调整
        contrast=0.2,               # 对比度调整
        saturation=0.2,             # 饱和度调整
        hue=0.1                     # 色相微调
    ),
    transforms.RandomRotation(15),   # ±15度随机旋转
    
    transforms.ToTensor(),           # 转为张量
    normalize,                      # 标准化
    
    # 可选：随机擦除(模拟遮挡)
    transforms.RandomErasing(
        p=0.5,                      # 50%概率应用
        scale=(0.02, 0.2),          # 擦除区域面积比例
        ratio=(0.3, 3.3),           # 宽高比范围
        value='random'              # 随机值填充
    )
])

In [21]:
class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files = None):
        super().__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [22]:
# Form train/valid dataloaders
train_set = FoodDataset(os.path.join(cfg['dataset_root'],"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True, num_workers=0, pin_memory=True)

valid_set = FoodDataset(os.path.join(cfg['dataset_root'], "validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=0, pin_memory=True)

One /kaggle/input/homework13-dataset/Food-11/training sample /kaggle/input/homework13-dataset/Food-11/training/0_0.jpg
One /kaggle/input/homework13-dataset/Food-11/validation sample /kaggle/input/homework13-dataset/Food-11/validation/0_0.jpg


In [23]:
import torch.nn as nn
import torch.nn.functional as F

def dwpw_conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    return nn.Sequential(
        # 深度可分离卷积 = 深度卷积 + 点卷积
        nn.Conv2d(in_channels, in_channels, kernel_size, 
                 stride=stride, padding=padding, groups=in_channels, bias=False),
        nn.BatchNorm2d(in_channels),
        nn.ReLU(inplace=True),
        
        # 点卷积(1x1卷积)
        nn.Conv2d(in_channels, out_channels, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

class StudentNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.cnn = nn.Sequential(
            # 初始卷积层(保持较高分辨率)
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            
            # 阶段1: 深度可分离卷积
            dwpw_conv(8, 16, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            
            # 阶段2: 增加通道数
            dwpw_conv(16, 32, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            
            # 阶段3: 进一步提取特征
            dwpw_conv(32, 64, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            
            # 阶段4: 最终特征提取
            dwpw_conv(64, 128, 3, stride=1, padding=1),
            
            # 全局平均池化适应不同输入尺寸
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        
        self.fc = nn.Sequential(
            nn.Dropout(0.2),  # 添加dropout防止过拟合
            nn.Linear(128, 11)
        )
        
        # 权重初始化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

def get_student_model():
    return StudentNet()

In [24]:
# DO NOT modify this block and please make sure that this block can run sucessfully. 
student_model = get_student_model()
summary(student_model, (3, 224, 224), device='cpu')
# You have to copy&paste the results of this block to HW13 GradeScope. 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 224, 224]             216
       BatchNorm2d-2          [-1, 8, 224, 224]              16
              ReLU-3          [-1, 8, 224, 224]               0
            Conv2d-4          [-1, 8, 224, 224]              72
       BatchNorm2d-5          [-1, 8, 224, 224]              16
              ReLU-6          [-1, 8, 224, 224]               0
            Conv2d-7         [-1, 16, 224, 224]             128
       BatchNorm2d-8         [-1, 16, 224, 224]              32
              ReLU-9         [-1, 16, 224, 224]               0
        MaxPool2d-10         [-1, 16, 112, 112]               0
           Conv2d-11         [-1, 16, 112, 112]             144
      BatchNorm2d-12         [-1, 16, 112, 112]              32
             ReLU-13         [-1, 16, 112, 112]               0
           Conv2d-14         [-1, 32, 1

In [10]:
# Load provided teacher model (model architecture: resnet18, num_classes=11, test-acc ~= 89.9%)
teacher_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False, num_classes=11)
# load state dict
teacher_ckpt_path = os.path.join(cfg['dataset_root'], "resnet18_teacher.ckpt")
teacher_model.load_state_dict(torch.load(teacher_ckpt_path, map_location='cpu'))
# Now you already know the teacher model's architecture. You can take advantage of it if you want to pass the strong or boss baseline. 
# Source code of resnet in pytorch: (https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py)
# You can also see the summary of teacher model. There are 11,182,155 parameters totally in the teacher model
# summary(teacher_model, (3, 224, 224), device='cpu')

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [25]:
def loss_fn_kd(student_logits, labels, teacher_logits, alpha=0.5, temperature=1.0):
    """
    知识蒸馏损失函数
    参数:
        student_logits: 学生模型的原始输出logits [batch_size, num_classes]
        labels: 真实标签 [batch_size]
        teacher_logits: 教师模型的原始输出logits [batch_size, num_classes]
        alpha: 蒸馏损失权重 (0-1之间)
        temperature: 温度参数(软化概率分布)
    返回:
        加权后的总损失
    """
    # 1. 计算常规交叉熵损失(学生模型输出与真实标签)
    ce_loss = F.cross_entropy(student_logits, labels)
    
    # 2. 计算KL散度损失(学生与教师模型的软化概率分布)
    # 对教师和学生输出应用softmax+log_softmax (带温度参数)
    soft_teacher = F.softmax(teacher_logits / temperature, dim=1)
    log_soft_student = F.log_softmax(student_logits / temperature, dim=1)
    
    # 计算KL散度并乘以温度平方(反向传播时梯度缩放补偿)
    kl_div_loss = F.kl_div(
        log_soft_student, 
        soft_teacher, 
        reduction='batchmean'
    ) * (temperature ** 2)
    
    # 3. 加权组合两种损失
    total_loss = (1.0 - alpha) * ce_loss + alpha * kl_div_loss
    
    return total_loss

In [26]:
# choose the loss function by the config
if cfg['loss_fn_type'] == 'CE':
    # For the classification task, we use cross-entropy as the default loss function.
    loss_fn = nn.CrossEntropyLoss() # loss function for simple baseline.

if cfg['loss_fn_type'] == 'KD': # KD stands for knowledge distillation
    loss_fn = loss_fn_kd # implement loss_fn_kd for the report question and the medium baseline.

# You can also adopt other types of knowledge distillation techniques for strong and boss baseline, but use function name other than `loss_fn_kd`
# For example:
# def loss_fn_custom_kd():
#     pass
# if cfg['loss_fn_type'] == 'custom_kd':
#     loss_fn = loss_fn_custom_kd

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
log(f"device: {device}")

# The number of training epochs and patience.
n_epochs = cfg['n_epochs']
patience = cfg['patience'] # If no improvement in 'patience' epochs, early stop

device: cuda


In [33]:
# Initialize a model, and put it on the device specified.
student_model.to(device)
teacher_model.to(device) # MEDIUM BASELINE

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(student_model.parameters(), lr=cfg['lr'], weight_decay=cfg['weight_decay']) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0.0

# teacher_model.eval()  # MEDIUM BASELINE
for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    student_model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []
    train_lens = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        with torch.no_grad():  # MEDIUM BASELINE
            teacher_logits = teacher_model(imgs)  # MEDIUM BASELINE
        
        logits = student_model(imgs)

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = loss_fn(logits, labels, teacher_logits) # MEDIUM BASELINE
        # loss = loss_fn(logits, labels) # SIMPLE BASELINE
        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=cfg['grad_norm_max'])

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels).float().sum()

        # Record the loss and accuracy.
        train_batch_len = len(imgs)
        train_loss.append(loss.item() * train_batch_len)
        train_accs.append(acc)
        train_lens.append(train_batch_len)
        
    train_loss = sum(train_loss) / sum(train_lens)
    train_acc = sum(train_accs) / sum(train_lens)

    # Print the information.
    log(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    student_model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []
    valid_lens = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = student_model(imgs)
            teacher_logits = teacher_model(imgs) # MEDIUM BASELINE

        # We can still compute the loss (but not the gradient).
        loss = loss_fn(logits, labels, teacher_logits) # MEDIUM BASELINE
        # loss = loss_fn(logits, labels) # SIMPLE BASELINE

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels).float().sum()

        # Record the loss and accuracy.
        batch_len = len(imgs)
        valid_loss.append(loss.item() * batch_len)
        valid_accs.append(acc)
        valid_lens.append(batch_len)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / sum(valid_lens)
    valid_acc = sum(valid_accs) / sum(valid_lens)

    # update logs
    
    if valid_acc > best_acc:
        log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        log(f"Best model found at epoch {epoch+1}, saving model")
        torch.save(student_model.state_dict(), f"{save_path}/student_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            log(f"No improvment {patience} consecutive epochs, early stopping")
            break
log("Finish training")
log_fw.close()

  0%|          | 0/157 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
# create dataloader for evaluation
eval_set = FoodDataset(os.path.join(cfg['dataset_root'], "evaluation"), tfm=test_tfm)
eval_loader = DataLoader(eval_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=0, pin_memory=True)

One /kaggle/input/homework13-dataset/Food-11/evaluation sample /kaggle/input/homework13-dataset/Food-11/evaluation/0000.jpg


In [31]:
# Load model from {exp_name}/student_best.ckpt
student_model_best = get_student_model() # get a new student model to avoid reference before assignment.
ckpt_path = f"{save_path}/student_best.ckpt" # the ckpt path of the best student model.
student_model_best.load_state_dict(torch.load(ckpt_path, map_location='cpu')) # load the state dict and set it to the student model
student_model_best.to(device) # set the student model to device

# Start evaluate
student_model_best.eval()
eval_preds = [] # storing predictions of the evaluation dataset

# Iterate the validation set by batches.
for batch in tqdm(eval_loader):
    # A batch consists of image data and corresponding labels.
    imgs, _ = batch
    # We don't need gradient in evaluation.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = student_model_best(imgs.to(device))
        preds = list(logits.argmax(dim=-1).squeeze().cpu().numpy())
    # loss and acc can not be calculated because we do not have the true labels of the evaluation set.
    eval_preds += preds

def pad4(i):
    return "0"*(4-len(str(i))) + str(i)

# Save prediction results
ids = [pad4(i) for i in range(0,len(eval_set))]
categories = eval_preds

df = pd.DataFrame()
df['Id'] = ids
df['Category'] = categories
df.to_csv(f"{save_path}/submission.csv", index=False) # now you can download the submission.csv and upload it to the kaggle competition.

  0%|          | 0/35 [00:00<?, ?it/s]